## CARGA DE LIBRERÍAS

In [1]:
# HTTP requests
import requests
# Date handler utility
import datetime
# Mark-Language parsing
import bs4
# DataFrame Manager
import pandas
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time
# Operating System management
import os

import selenium

from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import re
import numpy as np

## OBTENCIÓN DE LOS DATOS USANDO WEB SCRAPPING

In [2]:
USER_AGENT = ""

In [3]:
options = webdriver.ChromeOptions()

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

if USER_AGENT:
  options.add_argument(f"user-agent={USER_AGENT}")

driver = webdriver.Chrome(options=options)
     

In [4]:
driver.implicitly_wait(1)
driver.get("https://www.indomio.es/venta-casas/valencia-capital/")

In [5]:
links = driver.find_elements(By.TAG_NAME, 'a')

In [6]:
valids = []
# Iterate over link elements
for i in range(len(links)):
    href = links[i].get_attribute('href')
    if href and (re.search("^https://www.indomio.es/venta-casas/valencia-capital/[a-zA-Z-]+/$", href) is not None):
        valids.append(href)

In [7]:
links[8].get_attribute('href')

'https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/'

In [8]:
valids = valids[:19]

In [21]:
data = pd.DataFrame(columns = ["Title","Price","Characteristics", "Barrio"])

In [22]:
for link in valids:
    driver.get(link)
    barrio = re.match("^https://www.indomio.es/venta-casas/valencia-capital/([a-zA-Z-]+)/$",link)
    barrio = barrio.group(1)
    price = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardPrice')]
    barrio = list(np.full(len(price), barrio))
    title = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCard__title')] 
    characteristics =  [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardFeatureList')]
    aux_df = pd.DataFrame({"Title":title, "Price": price, "Characteristics":characteristics, "Barrio":barrio})
    data = pd.concat([data, aux_df], ignore_index=True)
    i = 2
    while len(price) >= 25:
        linknew = link + f'?pag={i}'
        print(linknew)
        driver.get(linknew)
        price = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardPrice')]
        if len(price) < 1:
            break
        barrio = re.match("^https://www.indomio.es/venta-casas/valencia-capital/([a-zA-Z-]+)/$",link)
        barrio = barrio.group(1)
        barrio = list(np.full(len(price), barrio))
        title = [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCard__title')]  
        characteristics =  [element.text for element in driver.find_elements(by = By.CLASS_NAME, value = 'in-reListCardFeatureList')]
        aux_df = pd.DataFrame({"Title":title, "Price": price, "Characteristics":characteristics, "Barrio":barrio})
        data = pd.concat([data, aux_df], ignore_index=True)
        i += 1
    

https://www.indomio.es/venta-casas/valencia-capital/sant-isidre-patraix-favara-vara-de-quart/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/sant-isidre-patraix-favara-vara-de-quart/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=4
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=5
https://www.indomio.es/venta-casas/valencia-capital/l-eixample/?pag=6
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/l-olivereta/?pag=4
https://www.indomio.es/venta-casas/valencia-capital/rascanya/?pag=2
https://www.indomio.es/venta-casas/valencia-capital/rascanya/?pag=3
https://www.indomio.es/venta-casas/valencia-capital/el-pla-del-real/?pag=2
https://www.indomio.es/ven

In [24]:
driver.close()

In [27]:
pd.DataFrame.to_csv(data, "data/WS_Indomio.csv")

## PRE-PRPCESADO DE LOS DATOS

In [ ]:
data = pd.read_csv("data/WS_Indomio.csv")

In [93]:
new_columns = data['Characteristics'].str.split('\n', expand=True)
column_names = ['hab', 'm2', 'bath', 'charact1', 'charact2', 'charact3']
new_columns.columns = column_names
data = pd.concat([data, new_columns], axis=1)
data.drop('Characteristics', axis=1, inplace=True)
data

,Title,Price,hab,m2,bath,charact1,charact2,charact3
0,Piso de una habitación Calle de la Linterna 19...,€ 220.000,1 habitación,60 m²,1 baño,Planta 3,Ascensor,Amueblado
1,"Piso a reformar, Russafa, València",€ 650.000,5 habitaciones,287 m²,3 baños,Ascensor,Terraza,None
2,"Piso Calle MORET, El Carme, València",€ 195.000,2 habitaciones,42 m²,1 baño,Planta 2,Lujo,No Ascensor
3,"Piso San Vicente Martir, Sant Francesc, València",€ 450.000,5 habitaciones,180 m²,2 baños,Planta 2,Ascensor,None
4,"Edificio Calle Quart, El Pilar, València",€ 10.000.000,1.320 m²,None,None,None,None,None
5,"Piso de tres habitaciones cuarta planta, La Xe...",€ 498.000,3 habitaciones,2 baños,Planta 4,Ascensor,None,None
6,"Piso de cuatro habitaciones cuarta planta, La ...",€ 1.000.000,4 habitaciones,3+ baños,Planta 4,Ascensor,None,None
7,"Edificio Calle de la Cultura, La Seu, València",€ 7.250.000,1.377 m²,None,None,None,None,None
8,"Piso de dos habitaciones buen estado, segunda ...",€ 420.000,2 habitaciones,85 m²,2 baños,Planta 2,Lujo,Ascensor
9,"Piso Calle de la Paz s/n, La Xerea, València",€ 825.000,3 habitaciones,200 m²,2 baños,Planta 2,Lujo,Ascensor
